In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
import random

def split_data(source_dir, train_dir, val_dir, val_ratio=0.15):
    # Create directories if they don't exist
    for directory in [train_dir, val_dir]:
        for subdir in ['NG', 'OK']:
            os.makedirs(os.path.join(directory, subdir), exist_ok=True)

    # Split the data
    for subdir in ['NG', 'OK']:
        files = os.listdir(os.path.join(source_dir, subdir))
        random.shuffle(files)

        n_total = len(files)
        n_val = int(n_total * val_ratio)

        val_files = files[:n_val]
        train_files = files[n_val:]

        # Copy files to validation set
        for file in val_files:
            shutil.copy(os.path.join(source_dir, subdir, file), os.path.join(val_dir, subdir, file))

        # Copy files to train set
        for file in train_files:
            shutil.copy(os.path.join(source_dir, subdir, file), os.path.join(train_dir, subdir, file))

# Define directories
source_dir = '/content/drive/MyDrive/Train'
train_dir = '/content/drive/MyDrive/Train/train'
val_dir = '/content/drive/MyDrive/Train/val'

# Split the dataset
split_data(source_dir, train_dir, val_dir)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

source_dir = '/content/drive/MyDrive/Train'
train_dir = '/content/drive/MyDrive/Train/train'
val_dir = '/content/drive/MyDrive/Train/val'
test_dir = '/content/drive/MyDrive/Test'

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  # b, 64, 16, 16
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # b, 128, 8, 8
            nn.ReLU(True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # b, 256, 4, 4
            nn.ReLU(True),
            nn.Flatten(),  # b, 256*4*4
            nn.Linear(256*4*4, 512),
            nn.ReLU(True),
            nn.Linear(512, 256),
        )
        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 256*4*4),
            nn.ReLU(True),
            nn.Unflatten(1, (256, 4, 4)),  # b, 256, 4, 4
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  # b, 128, 8, 8
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # b, 64, 16, 16
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),  # b, 3, 32, 32
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Prepare data
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Autoencoder().to(device)  # Move model to GPU if available
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with validation

# Initialize lists to store loss values
train_losses = []
val_losses = []

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, _ in train_loader:
        images = images.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, images)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Store the average training loss for this epoch
    train_losses.append(train_loss / len(train_loader))


    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, _ in val_loader:
            images = images.to(device)
            outputs = model(images)
            loss = criterion(outputs, images)
            val_loss += loss.item()

    # Store the average validation loss for this epoch
    val_losses.append(val_loss / len(val_loader))

    # Print epoch results
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'autoencoder.pth')

# Plot the loss curves
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()


# Load the trained model
model.load_state_dict(torch.load('autoencoder.pth'))
model.eval()

# Function to compute reconstruction error
def reconstruction_error(x, x_hat):
    return torch.mean((x - x_hat)**2, dim=(1, 2, 3))

# Compute reconstruction errors for test set
errors = []
labels = []
with torch.no_grad():
    for images, cls in test_loader:
        images = images.to(device)
        outputs = model(images)
        error = reconstruction_error(images, outputs)
        errors.extend(error.cpu().numpy())
        labels.extend(cls.cpu().numpy())

# Convert to numpy arrays
errors = np.array(errors)
labels = np.array(labels)

# Determine threshold (e.g., using the 95th percentile of 'OK' reconstruction errors)
ok_errors = errors[labels == 0]  # 0 is the label for 'OK' images
threshold = np.percentile(ok_errors, 95)

# Classify images as normal or anomalous
predictions = (errors > threshold).astype(int)

# Compute metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions)
recall = recall_score(labels, predictions)
f1 = f1_score(labels, predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")



# Compute the confusion matrix
cm = confusion_matrix(labels, predictions)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['OK', 'Defective'])
disp.plot(cmap=plt.cm.Blues)
disp.plot()
plt.title('Confusion Matrix')
plt.show()



# Calculate precision and recall for different thresholds
precision, recall, thresholds = precision_recall_curve(labels, errors)

# Plot the precision-recall curve
plt.figure(figsize=(10, 5))
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid(True)
plt.show()



In [ ]:
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, PrecisionRecallDisplay, roc_curve
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch

# Define transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load datasets
train_dir = '/content/drive/MyDrive/Train'
test_dir = '/content/drive/MyDrive/Test'

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Extract features from images
def extract_features(dataloader):
    features = []
    labels = []
    for images, batch_labels in dataloader:
        # Flatten the images
        batch_features = images.view(images.size(0), -1).numpy()
        features.append(batch_features)
        labels.extend(batch_labels.numpy())
    return np.vstack(features), np.array(labels)

# Extract features
X_train, y_train = extract_features(train_loader)
X_test, y_test = extract_features(test_loader)

# Train Isolation Forest
contamination = 0.1  #  expectation of anomaly ratio
clf = IsolationForest(contamination=contamination, random_state=42)
clf.fit(X_train)

# Predict anomalies
# Isolation Forest returns 1 for inliers and -1 for outliers, so we need to flip these
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

# Convert predictions to binary (0: normal, 1: anomaly)
y_pred_train = (y_pred_train == -1).astype(int)
y_pred_test = (y_pred_test == -1).astype(int)

# Assuming 'NG' (anomaly) is labeled as 1 and 'OK' (normal) as 0
y_true_train = (y_train == train_dataset.class_to_idx['NG']).astype(int)
y_true_test = (y_test == test_dataset.class_to_idx['NG']).astype(int)

# Compute metrics
accuracy = accuracy_score(y_true_test, y_pred_test)
precision = precision_score(y_true_test, y_pred_test)
recall = recall_score(y_true_test, y_pred_test)
f1 = f1_score(y_true_test, y_pred_test)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true_test, y_pred_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['OK', 'NG'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

# Precision-Recall Curve
precision_vals, recall_vals, thresholds = precision_recall_curve(y_true_test, clf.decision_function(X_test))
plt.figure(figsize=(8, 6))
plt.plot(recall_vals, precision_vals, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_true_test, clf.decision_function(X_test))
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
